In [1]:
import math
from itertools import product
import pandas as pd

class Minimization:
    def __init__(self):
        self.vect_bool = []
        self.count_var = 0
        self.before_pcnf = []
        self.before_pdnf = []
        self.implicants = []
        self.result_gluing = []
        self.all_combined_terms = []

    def read_vector_from_file(self, filename):
        try:
            with open(filename, "r") as file:
                self.vect_bool = file.read().strip()
                if not all(c in '01' for c in self.vect_bool):
                    raise ValueError("В файле должны быть только символы '0' и '1'.")
                self.count_var = int(math.log2(len(self.vect_bool)))
                if len(self.vect_bool) != 2 ** self.count_var:
                    raise ValueError("Длина вектора должна быть равна 2^n.")
        except FileNotFoundError:
            print(f"Файл {filename} не найден.")
            exit(1)
        except ValueError as e:
            print(f"Ошибка в файле: {e}")
            exit(1)

    def create_truth_table(self):
        self.truth_table = pd.DataFrame(
            list(product([0, 1], repeat=self.count_var)), 
            columns=[f"x{i+1}" for i in range(self.count_var)]
        )
        self.truth_table["F"] = list(map(int, self.vect_bool))

        for idx, row in self.truth_table.iterrows():
            if row["F"] == 0:
                self.before_pcnf.append(row[:-1].tolist())
            else:
                self.before_pdnf.append(row[:-1].tolist())

    def show_truth_table(self):
        print("\nТаблица истинности:")
        print(self.truth_table.to_string(index=False))

    def term_to_function(self, term, is_cnf=False):
        func = []
        for i, val in enumerate(term):
            var = f"x{i+1}"
            if val == 0:
                func.append(f"¬{var}" if not is_cnf else var)
            elif val == 1:
                func.append(var if not is_cnf else f"¬{var}")
        return " ∧ ".join(func)

    def show_nf(self):
        print("\nСКНФ:")
        sknf_terms = []
        for row in self.before_pcnf:
            sknf_terms.append(" ∨ ".join(
                f"¬x{i+1}" if val == 1 else f"x{i+1}" for i, val in enumerate(row)
            ))
        print(" * ".join(f"({term})" for term in sknf_terms) if sknf_terms else "1")

        print("\nСДНФ:")
        sdnf_terms = []
        for row in self.before_pdnf:
            sdnf_terms.append(" * ".join(
                f"x{i+1}" if val == 1 else f"¬x{i+1}" for i, val in enumerate(row)
            ))
        print(" ∨ ".join(f"({term})" for term in sdnf_terms) if sdnf_terms else "0")

    @staticmethod
    def can_combine(a, b):
        diff_count = 0
        result = a[:]
        for i in range(len(a)):
            if a[i] != b[i]:
                if a[i] == -1 or b[i] == -1:
                    return False, []
                diff_count += 1
                result[i] = -1
        return diff_count == 1, result

    def combine_terms(self, terms):
        new_terms = []
        used = set()
        for i in range(len(terms)):
            for j in range(i + 1, len(terms)):
                can_combine, combined = self.can_combine(terms[i], terms[j])
                if can_combine:
                    new_terms.append(combined)
                    used.add(i)
                    used.add(j)
        for i, term in enumerate(terms):
            if i not in used:
                new_terms.append(term)
        return new_terms

    def gluing(self):
        self.implicants = self.before_pdnf[:]
        while True:
            new_implicants = self.combine_terms(self.implicants)
            if new_implicants == self.implicants:
                break
            self.all_combined_terms.append(new_implicants)
            self.implicants = new_implicants
        self.result_gluing = self.implicants

    def build_implicant_matrix(self):
        implicant_labels = [self.term_to_function_string(implicant) for implicant in self.result_gluing]
        m_interms_labels = [self.term_to_function_string(term) for term in self.before_pdnf]

        matrix = []
        for implicant in self.result_gluing:
            row = []
            for term in self.before_pdnf:
                covers = all(
                    (implicant[k] == -1 or implicant[k] == term[k])
                    for k in range(len(term))
                )
                row.append(1 if covers else 0)
            matrix.append(row)

        implicant_df = pd.DataFrame(matrix, columns=m_interms_labels, index=implicant_labels)
        return implicant_df

    def minimize_cover(self, matrix):
        covered = [False] * len(matrix[0])
        result = []
        while not all(covered):
            max_cover, best_row = -1, -1
            for i in range(len(matrix)):
                cover_count = sum(
                    1 for j in range(len(matrix[i])) if matrix[i][j] and not covered[j]
                )
                if cover_count > max_cover:
                    max_cover = cover_count
                    best_row = i
            result.append(best_row)
            for j in range(len(matrix[0])):
                if matrix[best_row][j]:
                    covered[j] = True
        minimized_function = " ∨ ".join(
            f"({self.term_to_function(self.result_gluing[idx])})" for idx in result
        )
        return result, minimized_function

    def term_to_function_string(self, term):
        return " * ".join(
            f"x{i+1}" if bit == 1 else f"¬x{i+1}" if bit == 0 else ""
            for i, bit in enumerate(term)
        ).replace(" *", "").strip()

    def print_all_combined_terms(self):
        print("\nВсе этапы склеивания:")
        for step, terms in enumerate(self.all_combined_terms, start=1):
            print(f"\nШаг {step}:")
            for term in terms:
                print(f"({self.term_to_function_string(term)})")

    def print_implicants(self):
        print("\nИтоговые импликанты:")
        for term in self.result_gluing:
            print(f"({self.term_to_function_string(term)})")


if __name__ == "__main__":
    minimizer = Minimization()
    filename = "truth_table.txt"
    minimizer.read_vector_from_file(filename)
    minimizer.create_truth_table()
    minimizer.show_truth_table()
    minimizer.show_nf()
    minimizer.gluing()
    minimizer.print_all_combined_terms()

    implicant_df = minimizer.build_implicant_matrix()
    print("\nМатрица импликант (с функциями):")
    print(implicant_df.to_string(index=True))

    implicant_matrix = implicant_df.values
    minimized_cover, minimized_function = minimizer.minimize_cover(implicant_matrix)
    print("\nМинимизированная функция:")
    print(minimized_function)

    minimizer.print_implicants()



Таблица истинности:
 x1  x2  x3  x4  F
  0   0   0   0  0
  0   0   0   1  1
  0   0   1   0  0
  0   0   1   1  0
  0   1   0   0  0
  0   1   0   1  0
  0   1   1   0  0
  0   1   1   1  0
  1   0   0   0  0
  1   0   0   1  0
  1   0   1   0  0
  1   0   1   1  0
  1   1   0   0  1
  1   1   0   1  1
  1   1   1   0  1
  1   1   1   1  1

СКНФ:
(x1 ∨ x2 ∨ x3 ∨ x4) * (x1 ∨ x2 ∨ ¬x3 ∨ x4) * (x1 ∨ x2 ∨ ¬x3 ∨ ¬x4) * (x1 ∨ ¬x2 ∨ x3 ∨ x4) * (x1 ∨ ¬x2 ∨ x3 ∨ ¬x4) * (x1 ∨ ¬x2 ∨ ¬x3 ∨ x4) * (x1 ∨ ¬x2 ∨ ¬x3 ∨ ¬x4) * (¬x1 ∨ x2 ∨ x3 ∨ x4) * (¬x1 ∨ x2 ∨ x3 ∨ ¬x4) * (¬x1 ∨ x2 ∨ ¬x3 ∨ x4) * (¬x1 ∨ x2 ∨ ¬x3 ∨ ¬x4)

СДНФ:
(¬x1 * ¬x2 * ¬x3 * x4) ∨ (x1 * x2 * ¬x3 * ¬x4) ∨ (x1 * x2 * ¬x3 * x4) ∨ (x1 * x2 * x3 * ¬x4) ∨ (x1 * x2 * x3 * x4)

Все этапы склеивания:

Шаг 1:
(x1 x2 ¬x3)
(x1 x2  ¬x4)
(x1 x2  x4)
(x1 x2 x3)
(¬x1 ¬x2 ¬x3 x4)

Шаг 2:
(x1 x2)
(x1 x2)
(¬x1 ¬x2 ¬x3 x4)

Матрица импликант (с функциями):
                ¬x1 ¬x2 ¬x3 x4  x1 x2 ¬x3 ¬x4  x1 x2 ¬x3 x4  x1 x2 x3 ¬x4  x1 x2 x3 x4
x1 x2     